# Generating photometry from a ``Galaxy``

If you are working with a ``Galaxy`` (or many ``Galaxies``) with many different spectra you don't really want to have to generate photometry for each spectrum individually. Instead, you can use the galaxy level ``get_photo_luminosities`` and ``get_photo_fluxes`` methods to generate photometry for all spectra in the galaxy.

In [ ]:
from astropy.cosmology import Planck18 as cosmo
from synthesizer.emission_models import PacmanEmission
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.emission_models.dust.emission import Greybody
from synthesizer.filters import FilterCollection
from synthesizer.grid import Grid
from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG
from unyt import K

tau_v = 0.5
# dust curve slope
alpha = -1.0
dust_curve = PowerLaw(slope=alpha)
dust_emission_model = Greybody(30 * K, 1.2)

grid_dir = "../../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

# Create galaxy object
gal = load_CAMELS_IllustrisTNG(
    "../../../tests/data/",
    snap_name="camels_snap.hdf5",
    fof_name="camels_subhalo.hdf5",
    physical=True,
)[0]

# Get the model
pacman = PacmanEmission(
    grid=grid,
    tau_v=tau_v,
    dust_curve=dust_curve,
    dust_emission=dust_emission_model,
)

# Get the spectra
sed = gal.stars.get_spectra(pacman)

# Get fluxes
gal.get_observed_spectra(cosmo)

In [ ]:
# Get the filter collection
fs = [
    f"JWST/NIRCam.{f}"
    for f in ["F090W", "F150W", "F200W", "F277W", "F356W", "F444W"]
]

filters = FilterCollection(
    filter_codes=fs,
    new_lam=grid.lam,
)

# Get the photometry
gal.get_photo_luminosities(filters)
gal.get_photo_fluxes(filters)

The photometry produced by these methods are stored in the ``photo_luminosities`` and ``photo_fluxes`` attributes of the ``Galaxy.Stars`` object. These attributes are dictionaries containing the photometry for each spectra key.

In [ ]:
print(gal.stars.photo_luminosities)
print(gal.stars.photo_fluxes)

As before we can print the photometry.

In [ ]:
print(gal.stars.photo_fluxes["nebular"])

Or plot them.

In [ ]:
gal.stars.photo_fluxes["nebular"].plot_photometry(show=True)

In [ ]:
gal.stars.photo_fluxes["intrinsic"].plot_photometry(show=True)

In [ ]:
gal.stars.photo_fluxes["total"].plot_photometry(show=True)

## Calculating light radii

Once we have photometry we can calculate the radius enclosing a given fraction of the light for a component. Here we'll calculate the half light radius for both the intrinsic emission and the total emission in "F444W" in terms of luminosity, but before we can do that we need to get the particle spectra and call ``get_particle_photo_luminosities`` to first calculate the per particle photometry (above we used the galaxy level methods to calculate integrated spectra).

In [ ]:
# Get the particle spectra
gal.stars.get_particle_spectra(pacman)

# Get the particle photometry
gal.stars.get_particle_photo_luminosities(filters)

int_r50 = gal.stars.get_half_luminosity_radius(
    "intrinsic", "JWST/NIRCam.F444W"
)
tot_r50 = gal.stars.get_half_luminosity_radius("total", "JWST/NIRCam.F444W")
print(int_r50, tot_r50)

Similarly to the "attr" radii we can compute for any particle component, we can also compute the radius enclosing any fraction of the light for any particle component. 

In [ ]:
int_r20 = gal.stars.get_luminosity_radius(
    "intrinsic", "JWST/NIRCam.F444W", frac=0.2
)
tot_r20 = gal.stars.get_luminosity_radius(
    "total", "JWST/NIRCam.F444W", frac=0.2
)
int_r80 = gal.stars.get_luminosity_radius(
    "intrinsic", "JWST/NIRCam.F444W", frac=0.8
)
tot_r80 = gal.stars.get_luminosity_radius(
    "total", "JWST/NIRCam.F444W", frac=0.8
)
print(int_r20, int_r80, tot_r20, tot_r80)